# TVtoTV

Going to use the publicly available IMDB dataset for the tv shows. I have experience working with it from my movie to movie BFS solver, so hopefully the notebook work will be more or less trivial for this project.

If you would like to run this notebook yourself you need to download the tsvs from here: https://datasets.imdbws.com/ (they are updated daily)
and then put them in a folder in the root called "imdb-data"

You will need these tsvs:
name.basics.tsv.gz
title.basics.tsv.gz
title.principals.tsv.gz
title.ratings.tsv.gz

In [1]:
import json
import pandas as pd
from datetime import datetime

In [ ]:
# # Names data includes the nconst (id) for people and also their primary name
# names_data = pd.read_csv('./imdb-data/name.basics.tsv', sep='\t', header=0, usecols=['nconst', 'primaryName'])
# # Title data includes the tconst (id) for tv shows, movies along with their titles and start years
# titles_data = pd.read_csv('./imdb-data/title.basics.tsv', sep='\t', header=0, usecols=['tconst', 'titleType', 'primaryTitle', 'startYear'])
# # Principals data includes the associations between people and the works that they were featured in 
# # along with what role they worked (actor / actress are the roles that we are interested in)
# principals_data = pd.read_csv('./imdb-data/title.principals.tsv', sep='\t', header=0, usecols=['tconst', 'nconst', 'category'])
# # Ratings data includes all of the IMDB ratings, we are only interested in the number of votes to guage the popularity of a tv show
# ratings_data = pd.read_csv('./imdb-data/title.ratings.tsv', sep='\t', header=0, usecols=['tconst', 'numVotes'])

In [2]:
# Names data includes the nconst (id) for people and also their primary name
names_data = pd.read_csv('./imdb-data/name.basics.tsv', sep='\t', header=0)
# Title data includes the tconst (id) for tv shows, movies along with their titles and start years
titles_data = pd.read_csv('./imdb-data/title.basics.tsv', sep='\t', header=0)
# Principals data includes the associations between people and the works that they were featured in 
# along with what role they worked (actor / actress are the roles that we are interested in)
principals_data = pd.read_csv('./imdb-data/title.principals.tsv', sep='\t', header=0)
# Ratings data includes all of the IMDB ratings, we are only interested in the number of votes to guage the popularity of a tv show
ratings_data = pd.read_csv('./imdb-data/title.ratings.tsv', sep='\t', header=0)

/var/folders/zm/kglhp53x58752bpgh6114_f40000gn/T/ipykernel_12267/2487845291.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_data = pd.read_csv('./imdb-data/title.basics.tsv', sep='\t', header=0)


In [3]:
titles_data.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


In [14]:
titles_data.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [32]:
titles_data.loc[(titles_data.titleType == 'tvEpisode') & (titles_data.startYear == '2024')].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
245105,tt0256034,tvEpisode,La Forza del Destino,La Forza del Destino,0,2024,\N,180,"Music,Musical"
540359,tt0561593,tvEpisode,Kees van der Spek en Stine Jensen,Kees van der Spek en Stine Jensen,0,2024,\N,42,"Comedy,Game-Show"
969308,tt10003600,tvEpisode,Kevin Haddock,Kevin Haddock,0,2024,\N,46,"Adventure,Comedy,Fantasy"
984224,tt10030390,tvEpisode,Ptazeta,Ptazeta,0,2024,\N,\N,"Comedy,Talk-Show"
999176,tt10057290,tvEpisode,Pilot: The Death of A Police Chief's Son,Pilot: The Death of A Police Chief's Son,0,2024,\N,\N,"Crime,Drama"


In [34]:
titles_data.loc[(titles_data.titleType == 'tvSeries') & (titles_data.startYear == '2024')].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
973668,tt10011518,tvSeries,The Darkness,The Darkness,0,2024,\N,\N,Mystery
997929,tt10054992,tvSeries,D.O.P.E. Unit,D.O.P.E. Unit,0,2024,\N,\N,"Crime,Drama"
1008507,tt10074330,tvSeries,The Trades,The Trades,0,2024,\N,26,Comedy
1023036,tt10100318,tvSeries,Phoenix,Phoenix,0,2024,\N,\N,"Action,Drama,Sci-Fi"
1068550,tt10182942,tvSeries,Rumors,Rumors,0,2024,\N,\N,Drama


In [4]:
names_data.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0043044"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0069467,tt0050976"


In [5]:
principals_data.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N


In [ ]:
principals_data.category.unique

array(['self', 'director', 'producer', 'cinematographer', 'composer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'casting_director', 'archive_sound'],
      dtype=object)

In [6]:
ratings_data.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2113
1,tt0000002,5.6,285
2,tt0000003,6.4,2148
3,tt0000004,5.3,183
4,tt0000005,6.2,2871
